# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.251/13449/1 Dashboard: http://10.20.0.251:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,1018,Quinn,-0.214106,-0.322587
1,1040,Tim,-0.863415,0.608971
2,969,Dan,0.114130,-0.454055
3,966,Frank,-0.599144,-0.728896
4,996,Edith,0.704954,0.648928


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,9.999994e+02,-2.320816e-06,-2.076002e-04
std,3.161908e+01,5.771946e-01,5.775654e-01
min,8.440000e+02,-9.999992e-01,-9.999998e-01
25%,9.790000e+02,-4.929638e-01,-4.951169e-01
50%,1.000000e+03,7.697586e-03,4.330178e-03
75%,1.022000e+03,5.038608e-01,5.057660e-01
max,1.160000e+03,9.999999e-01,9.999993e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
952,Ursula,0.4506292091345929
992,Wendy,0.8073622955208448
1011,Dan,-0.7378103676249637
1066,Michael,-0.7794946980418203
1015,Alice,0.3026334133542379
1041,Patricia,0.08081873936832927
971,Tim,-0.8422181087789915
992,Frank,0.6507382725472803
1022,Michael,0.08653786857196555


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,1018,Quinn,-0.214106,-0.322587
1,1040,Tim,-0.863415,0.608971
2,969,Dan,0.114130,-0.454055
3,966,Frank,-0.599144,-0.728896
4,996,Edith,0.704954,0.648928


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function check_meta at 0x7fbfb619c9e0>, (<function apply at 0x7fbfdc3cbf80>, <function pandas_read_text at 0x7fbfb608a710>, [<function _make_parser_function.<locals>.parser_f at 0x7fbfbc2aa4d0>, (<function read_block_from_file at 0x7fbfb4c86680>, <OpenFile '/home/travis/build/dask/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', (<class 'dict'>, []), (<class 'dict'>, [['id', dtype('int64')], ['name', dtype('O')], ['x', dtype('float64')], ['y', dtype('float64')]]), ['id', 'name', 'x', 'y']], (<class 'dict'>, [['write_header', False], ['enforce', False], ['path', None]])), Empty DataFrame
Columns: [id, name, x, y]
Index: [], 'from_delayed'), 'from-delayed-11d955a74a1edf1acb4c512a3e454dd2'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,1018,Quinn,-0.214106,-0.322587
1,1040,Tim,-0.863415,0.608971
2,969,Dan,0.114130,-0.454055
3,966,Frank,-0.599144,-0.728896
4,996,Edith,0.704954,0.648928


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,952,Ursula,0.450629,NaN
1,992,Wendy,0.807362,NaN
2,1011,Dan,-0.737810,NaN
3,1066,Michael,-0.779495,NaN
4,1015,Alice,0.302633,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-f10c57f7-c68c-418b-8715-7b86879b4f08')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,1018,Quinn,-0.214106,-0.322587
1,1040,Tim,-0.863415,0.608971
2,969,Dan,0.114130,-0.454055
3,966,Frank,-0.599144,-0.728896
4,996,Edith,0.704954,0.648928
...,...,...,...,...
86395,1042,Ray,0.495498,-0.018391
86396,987,Ursula,0.594174,-0.214342
86397,1018,Edith,-0.379049,-0.771167
86398,1033,Alice,0.129724,0.699833


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-13.csv',
 'data/2000-01-02.csv',
 'data/2000-01-24.csv',
 'data/2000-01-04.csv',
 'data/2000-01-09.csv',
 'data/2000-01-17.csv',
 'data/2000-01-06.csv',
 'data/2000-01-08.csv',
 'data/2000-01-11.csv',
 'data/2000-01-28.csv',
 'data/2000-01-29.csv',
 'data/2000-01-18.csv',
 'data/2000-01-16.csv',
 'data/2000-01-05.csv',
 'data/2000-01-30.csv',
 'data/2000-01-20.csv',
 'data/2000-01-19.csv',
 'data/2000-01-21.csv',
 'data/2000-01-27.csv',
 'data/2000-01-12.csv',
 'data/2000-01-01.csv',
 'data/2000-01-07.csv',
 'data/2000-01-10.csv',
 'data/2000-01-25.csv',
 'data/2000-01-14.csv',
 'data/2000-01-26.csv',
 'data/2000-01-23.csv',
 'data/2000-01-03.csv',
 'data/2000-01-15.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-b7e83935-c933-438b-845e-97682f804327'),
 Delayed('read_data-6e3296c3-eb11-4232-8de6-3883b684844b'),
 Delayed('read_data-3b82a49b-180b-46f5-b37b-4887ce73fedb'),
 Delayed('read_data-40235c0c-24e8-44e6-bbe4-966124512995'),
 Delayed('read_data-50cf9c60-42d3-4260-892f-261b9574fead'),
 Delayed('read_data-2da9cc05-caa7-44dd-9dcd-35461cb3db38'),
 Delayed('read_data-9f08925f-daad-47d3-992c-d75362419670'),
 Delayed('read_data-a33ba865-8e8a-4ea0-a7bc-99a60a47964d'),
 Delayed('read_data-b7c651cf-0df5-4157-ae4c-0899bd752a0c'),
 Delayed('read_data-8557b9ee-6ab5-4084-8c8e-5f05a2225b74'),
 Delayed('read_data-6344e4c6-8e17-4001-b98f-931bf8e6725d'),
 Delayed('read_data-91998419-6cc9-4517-a25c-60783e916611'),
 Delayed('read_data-f30334fc-b31b-4ee2-be4b-af23eb2a97f8'),
 Delayed('read_data-94b37894-4c15-4780-9abf-4ac55a2c349b'),
 Delayed('read_data-64beb63a-3233-452e-b040-5dd100e45481'),
 Delayed('read_data-95f654ab-57d5-4d31-8ff9-5867c0430bc1'),
 Delayed('read_data-920311cf-c117-4989-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1041,Yvonne,0.463499,0.368413
1,1001,Wendy,0.958907,0.722575
2,1046,Patricia,0.435074,-0.318281
3,991,Alice,-0.850399,-0.323550
4,1008,George,0.579771,-0.822412


In [26]:
df.describe().compute()

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,9.999989e+02,1.041765e-04,-1.630995e-04
std,3.162035e+01,5.772541e-01,5.775452e-01
min,8.440000e+02,-9.999992e-01,-9.999998e-01
25%,9.790000e+02,-4.929638e-01,-4.951169e-01
50%,1.000000e+03,7.697586e-03,4.330178e-03
75%,1.022000e+03,5.038608e-01,5.057660e-01
max,1.160000e+03,9.999999e-01,9.999993e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.